In [1]:
import numpy as np

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

## Step 0 : Load the Data

In [35]:
x = np.load('x.npy') # 5000 images
y = np.load('y.npy')

theta_0 = np.load('theta_0.npy')
theta_1 = np.load('theta_1.npy')

## Step 1. Convert Pixels Using Fixed Point Notation

In [ ]:
def float_to_fix_point_unsigned(x: float, n: int) -> int:
    if x >= 1.0:
        x = 0.99999
    res = int(x * pow(2, n))
    return res

imgIndex = 0
# image [0.0, 1.0[
# 4 bits [0 signed, 0 integer, 4 fractional]
image = x[imgIndex]

image_fp04 = np.zeros_like(image)
for i in range(image.shape[0]):
    image_fp04[i] = float_to_fix_point_unsigned(image[i], 4)

## Step 2. Convert First Hidden Layer Weights Into Fixed Point Notation

In [ ]:
# theta_0 [-1.463369, 1.00899]
# 4 bits [1 signed, 2 integer, 1 fractional]
weights_fp04 = np.zeros_like(theta_0)
for i in range(theta_0.shape[0]):
    weights_fp04[i] = float_to_fix_point_unsigned(theta_0[i], 4)

print(weights_fp04[::40])

1.0089920104197974


In [ ]:
xp = np.hstack((1, image))
hiddenLayer1 = np.dot(xp,theta_0.T)

# Step 3. Make The Second Hidden Layer Multiplication X*W 
fl_hiddenLayer1 = []
for i in range(0, len(theta_0)): # [0, 24]
    row = theta_0[i]
    product = 0
    for j in range(0, len(row)): # [0, 400]
        weight = row[j]
        pixel = xp[j]
        # todo convertir les valeurs en nombres de 4 bits signés?
        product = product + weight * pixel
    fl_hiddenLayer1.append(product)

# Step 4. Apply the Sigmoid Function For First Hidden Layer

# Step 5. Convert Second Hidden Layer Weights Into Fixed Point Notation

# Step 6. Make The Second Hidden Layer Multiplication X*W 

# Step 7. Apply the Sigmoid Function For Second Hidden Layer

# Step 8. Display FPGA Approximation

In [ ]:
# Calculer sur le FPGA
xp = np.hstack((1, image)) # Ajouter 1 pour le biais
hiddenLayer1 = np.dot(xp,theta_0.T)

a = sigmoid(hiddenLayer1) # Sortie de la couche 1
ap = np.hstack((1, a)) # Ajouter 1 pour le biais
b = sigmoid(np.dot(ap,theta_1.T)) # Sortie de la couche 2
print("FPGA Output:",b)

# Calculer dans le notebook
pred = b.argmax()
print("Prediction:", pred, "Label:", y[idx])